In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from keras.utils import to_categorical

# Load trained model
model = load_model('model_hand.h5')

# Load word dictionary
labels = ['hello', 'world', 'goodbye', 'thanks', 'please']
word_dict = {i: label for i, label in enumerate(labels)}

# Read in test image
img_path = "test_image.png"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# Preprocess image
img_copy = cv2.GaussianBlur(img, (7, 7), 0)
img_gray = cv2.cvtColor(img_copy, cv2.COLOR_GRAY2BGR)
thresh = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Make predictions on each contour
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w < 10 or h < 10:
        continue
    roi = thresh[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (28, 28))
    roi_flattened = roi_resized.reshape(1, 28, 28)
    roi_normalized = roi_flattened / 255.0
    pred = model.predict(roi_normalized)
    pred_label = word_dict[np.argmax(pred)]
    print("Predicted label: {}".format(pred_label))

# Display the preprocessed image with bounding boxes around recognized words
img_draw = img.copy()
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    if w < 10 or h < 10:
        continue
    cv2.rectangle(img_draw, (x, y), (x+w, y+h), (255, 0, 0), 1)

cv2.imshow('Test Image', img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()